In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import time
import requests
import hmac
import os
import json
from dotenv.main import load_dotenv
from hashlib import sha256

In [5]:
load_dotenv("./dev.env")
api_url="https://open-api.bingx.com"
api_key=os.getenv("api_key")
secret_key=os.getenv("secret_key")

In [27]:
def data_mining(crypto_pair, interval, start_time, end_time):
    payload = {}
    path = '/openApi/swap/v2/quote/klines'
    method = "GET"
    start_timestamp=int(time.mktime(datetime.datetime.strptime(start_time,"%Y/%m/%d").timetuple())*1000)
    end_timestamp=int(time.mktime(datetime.datetime.strptime(end_time,"%Y/%m/%d").timetuple())*1000)
    timestamps=[]
    data=[]
    
    for i in range(start_timestamp, end_timestamp, 432000000):
        timestamps.append(i)
        
    for j in range(len(timestamps)-1):
        paramsMap = {
            "symbol": crypto_pair,
            "interval": interval,
            "startTime": timestamps[j],
            "endTime": timestamps[j+1],
            "limit": 1440
        }
        paramsStr = praseParam(paramsMap)
        data_dict=json.loads(send_request(method, path, paramsStr, payload))['data']
        data_frame=pd.DataFrame(data_dict)
        data.append(data_dict)
        
    return data
    
def get_sign(api_secret, payload):
    signature = hmac.new(api_secret.encode("utf-8"), payload.encode("utf-8"), digestmod=sha256).hexdigest()
    return signature


def send_request(method, path, urlpa, payload):
    url = "%s%s?%s&signature=%s" % (api_url, path, urlpa, get_sign(secret_key, urlpa))
    print(url)
    headers = {
        'X-BX-APIKEY': api_key,
    }
    response = requests.request(method, url, headers=headers, data=payload)
    return response.text

def praseParam(paramsMap):
    sortedKeys = sorted(paramsMap)
    paramsStr = "&".join(["%s=%s" % (x, paramsMap[x]) for x in sortedKeys])
    return paramsStr+"&timestamp="+str(int(time.time() * 1000))

def data_preprocessing(data):
    df=pd.DataFrame(data, )

def milisec_to_sec(x):
    return x/1000

def getting_time_index(df):
    df['time'].apply(milisec_to_sec)
    df['time'].apply(datetime.datetime.fromtimestamp)
    df.set_index('time')
    df.drop(['time'])
    return df

In [28]:
data_test=data_mining("ETH-USDT", "5m", "2023/01/01", "2023/01/31")

https://open-api.bingx.com/openApi/swap/v2/quote/klines?endTime=1672950600000&interval=5m&limit=1440&startTime=1672518600000&symbol=ETH-USDT&timestamp=1691262884862&signature=6c93d13826eea3462595d02aaf98d360e57c547c95721e2cc61cfdd4f09f72dd
https://open-api.bingx.com/openApi/swap/v2/quote/klines?endTime=1673382600000&interval=5m&limit=1440&startTime=1672950600000&symbol=ETH-USDT&timestamp=1691262885948&signature=787ed99ad9e8adf405c09e30dcf4e84287a246c7e09adab1c02de4ef0ff1d2d4
https://open-api.bingx.com/openApi/swap/v2/quote/klines?endTime=1673814600000&interval=5m&limit=1440&startTime=1673382600000&symbol=ETH-USDT&timestamp=1691262886812&signature=e4c17d19f4bfb16b05cf9fd305712f5b986ab823f4166292308a2efbf88f45f7
https://open-api.bingx.com/openApi/swap/v2/quote/klines?endTime=1674246600000&interval=5m&limit=1440&startTime=1673814600000&symbol=ETH-USDT&timestamp=1691262887551&signature=01bdb845971f710fe6fd2c53a55a9f1d0fec4f83f3959e067da04096c6bd81c0
https://open-api.bingx.com/openApi/swap/

In [22]:
data_test

[[{'open': '1202.40',
   'close': '1202.55',
   'high': '1202.75',
   'low': '1202.35',
   'volume': '423.24',
   'time': 1672518600000},
  {'open': '1202.65',
   'close': '1202.15',
   'high': '1202.65',
   'low': '1201.80',
   'volume': '299.21',
   'time': 1672518900000},
  {'open': '1202.15',
   'close': '1202.85',
   'high': '1202.95',
   'low': '1202.00',
   'volume': '427.05',
   'time': 1672519200000},
  {'open': '1202.85',
   'close': '1203.00',
   'high': '1203.20',
   'low': '1202.80',
   'volume': '879.34',
   'time': 1672519500000},
  {'open': '1203.00',
   'close': '1201.75',
   'high': '1203.00',
   'low': '1201.75',
   'volume': '289.37',
   'time': 1672519800000},
  {'open': '1201.75',
   'close': '1201.85',
   'high': '1201.95',
   'low': '1201.75',
   'volume': '502.18',
   'time': 1672520100000},
  {'open': '1201.85',
   'close': '1202.00',
   'high': '1202.45',
   'low': '1201.70',
   'volume': '348.74',
   'time': 1672520400000},
  {'open': '1201.90',
   'close': 